# Generador de personas aleatorias

In [68]:
import random as rm
import numpy as np
import pandas as pd
from faker import Faker
import datetime as dt
import pytz
import re
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder
from funciones import quitar_acentos
import urllib.parse
from typing import Union

In [85]:
def sacar_curp(nombre: str, apellido_paterno: str, apellido_materno: str, sexo: str, fecha_nac: Union[str, dt.datetime], estado_nac: str) -> str:
	nombre = quitar_acentos(nombre).upper().replace('Ñ', 'X')
	apellido_paterno = quitar_acentos(apellido_paterno).upper().replace('Ñ', 'X')
	apellido_materno = quitar_acentos(apellido_materno).upper().replace('Ñ', 'X')
	estado_nac = quitar_acentos(estado_nac)
	if isinstance(fecha_nac, str): fecha_nac = dt.datetime.strptime(fecha_nac, '%Y-%m-%d')
	# Primera letra y vocal del primer apellido
	curp = apellido_paterno[0:2].upper()
	# Primera letra del segundo apellido
	curp += apellido_materno[0:1].upper()
	# Primera letra del nombre de pila
	curp += nombre[0:1].upper()
	# Fecha de nacimiento (2 últimos dígitos del año, 2 del mes y 2 del día de nacimiento)
	curp += fecha_nac.strftime('%y%m%d')
	# Letra del sexo (H o M)
	curp += sexo[0:1].upper()
	# Dos letras correspondientes a la entidad de nacimiento (Sonora => SR)
	# en el caso de extranjeros, se marca como NE (Nacido Extranjero)
	clave_estados = {
		'aguascalientes': 'AS',
		'baja_california': 'BC',
		'baja_california_sur': 'BS',
		'campeche': 'CC',
		'coahuila': 'CL',
		'colima': 'CM',
		'chiapas': 'CS',
		'chihuahua': 'CH',
		'distrito_federal': 'DF',
		'ciudad_de_mexico': 'DF',
		'durango': 'DG',
		'guanajuato': 'GT',
		'guerrero': 'GR',
		'hidalgo': 'HG',
		'jalisco': 'JC',
		'mexico': 'MC',
		'estado_de_mexico': 'MC',
		'michoacan': 'MN',
		'morelos': 'MS',
		'nayarit': 'NT',
		'nuevo_leon': 'NL',
		'oaxaca': 'OC',
		'puebla': 'PL',
		'queretaro': 'QT',
		'quintana_roo': 'QR',
		'san_luis_potosi': 'SP',
		'sinaloa': 'SL',
		'sonora': 'SR',
		'tabasco': 'TC',
		'tamaulipas': 'TS',
		'tlaxcala': 'TL',
		'veracruz': 'VZ',
		'yucatan': 'YN',
		'zacatecas': 'ZS'
	}
	estado_nac_key = estado_nac.replace(' ', '_').lower()
	if estado_nac_key in clave_estados:
		clave_estado_nac = clave_estados[estado_nac_key]
	else:
		clave_estado_nac = 'NE'
	curp += clave_estado_nac
	# Primera consonante interna del primer apellido
	curp += re.sub(f'[aeiou]', '', apellido_paterno)[1:2].upper()
	# Primera consonante interna del segundo apellido
	curp += re.sub(f'[aeiou]', '', apellido_materno)[1:2].upper()
	# Primera consonante interna del nombre
	curp += re.sub(f'[aeiou]', '', nombre)[1:2].upper()
	# Dígito verificador del 0-9 para fechas de nacimiento hasta el año 1999 y A-Z para fechas de nacimiento a partir del 2000
	int_char = [rm.randint(0, 9), chr(rm.randint(65, 90))]
	curp += str(int_char[0] if fecha_nac.year <= 1999 else int_char[1])
	# Homoclave, para evitar duplicaciones
	curp += str(rm.randint(0, 9))
	return curp

def obtener_coordenadas(ciudad, estado, pais = 'mexico'):
	# Hacer peticion a un api para obtener coordenadas del centro y extremas
	querystring = urllib.parse.urlencode({'city': ciudad, 'state': estado, 'country': pais})
	df = pd.read_json(f'https://nominatim.openstreetmap.org/search?{querystring}&polygon_geojson=1&format=json')
	# Obtener coordenadas del centro
	centro = (df['lat'][0], df['lon'][0])
	# Obtener extremos originales
	minx, maxx, miny, maxy = [float(i) for i in df['boundingbox'][0]]
	ne, no, se, so = [(maxx, maxy), (maxx, miny), (minx, maxy), (minx, miny)]
	return {'centro': centro, 'noreste': ne, 'noroeste': no, 'sureste': se, 'suroeste': so}

def generar_puntos_random(centro, extremo_so, extremo_ne, n = 1, reducir = True):
	# Obtener coordenadas del centro
	cenx, ceny = centro
	# Obtener extremos originales
	minx, miny = extremo_so
	maxx, maxy = extremo_ne

	# Hallar punto medio entre (suroeste centro) y (centro noreste)
	minx, miny = ((minx + cenx) / 2), ((miny + ceny) / 2)
	maxx, maxy = ((cenx + maxx) / 2), ((ceny + maxy) / 2)
	# True para reducir el area de la ciudad
	if reducir:
		# Hallar punto medio de puntos medios y centro
		minx, miny = ((minx + cenx) / 2), ((miny + ceny) / 2)
		maxx, maxy = ((cenx + maxx) / 2), ((ceny + maxy) / 2)
	# Obtener n puntos aleatorios dentro del area
	puntos_random = []
	while len(puntos_random) < n:
		puntos_random.append((rm.uniform(minx, maxx), rm.uniform(miny, maxy)))
	return puntos_random

def buscar_direccion(puntos_random):
	geolocator = Nominatim(user_agent = 'geoapiExercises')
	dirr = {}
	i = 0
	for punto_random in puntos_random:
		listo = False
		direccion = geolocator.reverse(punto_random, language = 'es_MX')
		direccion = direccion[0].split(', ')
		if len(direccion) == 9:
			print('\033[34mHay de 9\033[0m')
			print(list(direccion))
		elif len(direccion) == 8:
			dirr['local'] = direccion[0]
			dirr['numero'] = direccion[1]
			dirr['calle'] = direccion[2]
			dirr['colonia'] = direccion[3]
			dirr['cp'] = direccion[-2]
			listo = True
		elif len(direccion) == 7:
			dirr['local'] = direccion[0]
			dirr['calle'] = direccion[1]
			dirr['numero'] = rm.randint(1, 999)
			dirr['colonia'] = direccion[2]
			dirr['cp'] = direccion[-2]
			listo = True
		elif len(direccion) == 6:
			dirr['calle'] = direccion[0]
			dirr['numero'] = rm.randint(1, 999)
			dirr['colonia'] = direccion[1]
			dirr['cp'] = direccion[-2]
			listo = True
		else:
			print('No se encontro calle o colonia en', punto_random)
		dirr['ciudad'] = direccion[-4]
		dirr['estado'] = direccion[-3]
		dirr['pais'] = direccion[-1]
		if listo: break
		i += 1
	return dirr

In [84]:
file_ciudades = './catalogos/ciudades.csv'
ct_ciudades = pd.read_csv(file_ciudades)

ct_ciudades['centro'] = '(cenx, ceny)'
ct_ciudades[['latitud', 'longitud', 'centro']]

,latitud,longitud,centro
0,NaN,NaN,"(cenx, ceny)"
1,NaN,NaN,"(cenx, ceny)"
2,NaN,NaN,"(cenx, ceny)"
3,21.849624,-102.712543,"(cenx, ceny)"
4,NaN,NaN,"(cenx, ceny)"
...,...,...,...
2946,NaN,NaN,"(cenx, ceny)"
2947,NaN,NaN,"(cenx, ceny)"
2948,NaN,NaN,"(cenx, ceny)"
2949,NaN,NaN,"(cenx, ceny)"


In [81]:
reap = pd.read_csv('./conjunto_datos/relacion_edad_altura_peso.csv')
nombres = pd.read_csv('./catalogos/nombres.csv')
profesiones = pd.read_csv('./catalogos/profesiones.csv')
file_ciudades = './catalogos/ciudades.csv'
ct_ciudades = pd.read_csv(file_ciudades)

# file_direcciones = './catalogos/direcciones.csv'
# ct_direcciones = pd.read_csv(file_direcciones)

def personas_aleatorias(n = 5):
	fake = Faker(['es_MX'])
	personas = []
	for _ in range(n):
		sexo = rm.choice(['H', 'M'])
		# generar tipo de sangre y asignar probabilidad a cada valor para que salga
		tipo_sangre = rm.choices(['A+', 'A-', 'B+', 'B-', 'AB+', 'AB-', 'O+', 'O-'], weights = [30, 6, 9, 2, 4, 1, 39, 9])
		tipo_sangre = tipo_sangre[0]

		fecha_nac = fake.date_of_birth(minimum_age = int(reap['edad'].min()))
		# calcular edad en base a la fecha de nacimiento
		dia_actual = dt.date.today()
		edad = dia_actual.year - fecha_nac.year - ((dia_actual.month, dia_actual.day) < (fecha_nac.month, fecha_nac.day))

		# dependiendo de la edad se busca el margen de alturas y pesos que tenga esa edad
		# si es mayor que la edad maxima que este registrada se le da el margen de esa edad maxima
		# mucha gente deja de crecer a los 20
		margen_ap = reap[(reap['edad'] == edad) if edad <= reap['edad'].max() else (reap['edad'] == reap['edad'].max())].reset_index()

		# dependiendo del sexo se generan nombres femeninos o masculinos
		# y estaturas y pesos dentro de su margen correspondiente
		if sexo == 'H':
			titulo = rm.choice(['Sr.', 'Dr.', 'Mtro.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
			nombre = rm.choice(nombres['h'])
			altura = round(rm.uniform(margen_ap['lim_inf.altura.h.cm'][0], margen_ap['lim_sup.altura.h.cm'][0]), 2)
			peso = round(rm.uniform(margen_ap['lim_inf.peso.h.kg'][0], margen_ap['lim_sup.peso.h.kg'][0]), 2)
		else:
			titulo = rm.choice(['Sra.', 'Dra.', 'Mtra.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
			nombre = rm.choice(nombres['m'])
			altura = round(rm.uniform(margen_ap['lim_inf.altura.m.cm'][0], margen_ap['lim_sup.altura.m.cm'][0]), 2)
			peso = round(rm.uniform(margen_ap['lim_inf.peso.m.kg'][0], margen_ap['lim_sup.peso.m.kg'][0]), 2)

		apellido_paterno = fake.last_name()
		apellido_materno = fake.last_name()

		id_estado = rm.choice(ct_ciudades['id_estado'].unique())
		ct_ciudad = ct_ciudades[ct_ciudades['id_estado'] == id_estado].sample()

		ciudad = ct_ciudad['ciudad'].values[0]
		estado = ct_ciudad['estado'].values[0]

		tel = f'({ct_ciudad["clave_lada"].values[0]})' + str(rm.randint(1000000, 9999999))

		curp = sacar_curp(nombre, apellido_paterno, apellido_materno, sexo, fecha_nac, estado)

		# Generar un nombre de usuario para un email
		switcher = {
			1: fake.user_name(),
			# aperez
			2: f'{nombre[0]}{apellido_paterno}',
			# alonso_perez
			3: f'{nombre}_{apellido_paterno}',
			# aps70
			4: f'{nombre[0]}{apellido_paterno[0]}{apellido_materno[0]}{fecha_nac.strftime("%y")}',
			# aps1970
			5: f'{nombre[0]}{apellido_paterno[0]}{apellido_materno[0]}{fecha_nac.year}',
			# alonsoperezsoltero
			6: f'{nombre}{apellido_paterno}{apellido_materno}'
		}
		rand_int = rm.randint(min(list(switcher.keys())), max(list(switcher.keys())))
		usuario = switcher.get(rand_int, 'usuario')
		# quitar espacios o reemplazarlos por un _
		usuario = usuario.replace(' ', rm.choice(['', '_'])).lower()
		usuario = quitar_acentos(usuario)
		subdominio = rm.choice(['.com', '.net', '.io', '.gob', '.org', '.edu'])
		correo = usuario + f'@example{subdominio}'

		profesion = profesiones.sample().values[0][0] if edad >= 18 else None
		"""
		El Número de Seguridad Social (NSS) es una homoclave que asigna el IMSS
		para llevar un registro de los trabajadores y asegurados
		que están inscritos en dicha institución.
		El NSS es único, permanente e intransferible.
		"""
		# El NSS está compuesto por 11 dígitos:
		if profesion:
			# Los primeros dos caracteres están vinculados a la subdelegación en el que la persona fue afiliada.
			nss1 = ('%02d' % rm.randint(0, 99))
			# Los dos dígitos siguientes indican el año en el que la persona se afilió al Seguro Social.
			nss2 = ('%02d' % rm.randint(0, 99))
			# Los siguientes dos dígitos corresponden a la fecha de nacimiento del afiliado.
			nss3 = fecha_nac.strftime('%y')
			# Los cuatro números siguientes son los dígitos que asigna el IMSS al trabajador.
			nss4 = ('%02d' % rm.randint(0, 9999))
			# El último dígito corresponde al número de verificación del trabajador en el IMSS.
			nss5 = str(rm.randint(0, 9))
			nss = f'{nss1} {nss2} {nss3} {nss4} {nss5}'
		else:
			nss = None

		"""Coordenadas de la ciudad"""
		# si el catalogo ya tiene coordenadas, tomar esos valores
		if pd.notna(ct_ciudad['latitud'].values[0]) and pd.notna(ct_ciudad['longitud'].values[0]):
			lat = ct_ciudad['latitud'].values[0]
			lon = ct_ciudad['longitud'].values[0]
			# lat, lon = ct_ciudad['centro'].values[0]
			# suroeste = ct_ciudad['suroeste'].values[0]
			# noreste = ct_ciudad['noreste'].values[0]
			suroeste = noreste = None
		else:
			# si no, buscar en internet el punto central y extremos segun la ciudad
			try:
				coordenadas = obtener_coordenadas(ciudad, estado)
				lat, lon = coordenadas['centro']
				suroeste = coordenadas['suroeste']
				noreste = coordenadas['noreste']
				# guardar coordenadas en el catalogo
				# para que a la otra no haya que buscar en internet
				ct_ciudad_index_row = ct_ciudad.index.values[0]
				# ct_ciudades.loc[ct_ciudad_index_row, 'centro'] = coordenadas['centro']
				# ct_ciudades.loc[ct_ciudad_index_row, 'suroeste'] = coordenadas['suroeste']
				# ct_ciudades.loc[ct_ciudad_index_row, 'noreste'] = coordenadas['noreste']
				ct_ciudades.loc[ct_ciudad_index_row, 'latitud'] = lat
				ct_ciudades.loc[ct_ciudad_index_row, 'longitud'] = lon
				ct_ciudades.to_csv(file_ciudades, index = False)
			except BaseException as be:
				# En caso de no haber encontrado o no tener internet
				print('\033[31mException:\033[0m')
				print(be)
				lat = lon = np.NaN
				suroeste = noreste = None

		"""Direccion"""
		suroeste = noreste = None
		if not np.isnan(lat) and not np.isnan(lon) and suroeste and noreste:
			puntos_random = generar_puntos_random((lat, lon), suroeste, noreste, 5)
			try:
				direccion = buscar_direccion(puntos_random)
				calle = direccion['calle']
				numero = direccion['numero']
				cp = direccion['cp']
			# print(pd.DataFrame(direccion))
			except BaseException as be:
				print('\033[31mException:\033[0m')
				print(be)
				calle = numero = cp = None
		else:
			calle = numero = cp = None

		"""Zona horaria"""
		if not np.isnan(lat) and not np.isnan(lon):
			# obtener zona horaria segun latitud y longitud
			obj = TimezoneFinder()
			timezone = obj.timezone_at(lat = lat, lng = lon)
			# 'datetime.now' tiene DST (Daylight Saving Time)
			tz_offset = dt.datetime.now(pytz.timezone(timezone)).strftime('%z')
		# en Enero no hay DST
		#tz_offset = pytz.timezone(timezone).localize(dt.datetime(2011, 1, 1)).strftime('%z')
		else:
			timezone = tz_offset = None

		personas.append({
			'titulo': titulo,
			'nombre': nombre,
			'apellido_paterno': apellido_paterno,
			'apellido_materno': apellido_materno,
			'fecha_nac': fecha_nac.strftime('%Y-%m-%d'),
			'sexo': sexo,
			'altura': altura,
			'peso': peso,
			'tipo_sangre': tipo_sangre,
			'correo': correo,
			'tel': tel,
			'calle': calle,
			'numero': numero,
			'cp': cp,
			'ciudad': ciudad,
			'estado': estado,
			'lat': lat,
			'lon': lon,
			'tz_offset': tz_offset,
			'zona_horaria': timezone,
			'profesion': profesion,
			'curp': curp,
			'nss': nss,
			'color_fav': fake.safe_color_name(),
		})
	return pd.DataFrame(personas)

personas_aleatorias(10)

Exception:
'lat'


,titulo,nombre,apellido_paterno,apellido_materno,fecha_nac,sexo,altura,peso,tipo_sangre,correo,...,ciudad,estado,lat,lon,tz_offset,zona_horaria,profesion,curp,nss,color_fav
0,Dr.,Emilio,Paz,Olivárez,1908-06-15,H,168.33,82.84,O-,epo1908@example.edu,...,Zacapalco,Guerrero,18.537217,-99.443243,-0600,America/Mexico_City,Director de Marketing,PAOE080615HGRALM86,74 01 08 4141 6,green
1,None,Karla,Moreno,Carrero,1916-01-14,M,173.23,57.17,A+,kmoreno@example.org,...,Tabasco,Zacatecas,21.865270,-102.913063,-0600,America/Mexico_City,Científico clínico histocompatibilidad e inmun...,MOCK160114MZSOAA28,67 23 16 7906 8,navy
2,Mtro.,Yandel,Marín,Calderón,1916-05-08,H,172.27,71.56,A-,ymarin@example.net,...,Antonio Amaro,Durango,24.276728,-104.017861,-0600,America/Monterrey,Operador de cámara,MACY160508HDGAAA28,18 78 16 1680 2,teal
3,None,Berta,Ornelas,Limón,2013-06-15,M,123.74,35.35,A-,bornelas@example.com,...,Guatimape,Durango,24.807350,-104.919513,-0600,America/Monterrey,None,ORLB130615MDGRIEU8,None,yellow
4,Mtra.,Ariadna,Orozco,Carrasco,1962-05-20,M,173.48,65.46,O+,ariadnaorozcocarrasco@example.net,...,Zapotitlán Salinas,Puebla,18.329224,-97.473600,-0600,America/Mexico_City,Comerciante de bonos,ORCA620520MPLRAR93,34 06 62 1449 1,maroon
5,None,Gonzalo,Carrero,Zúñiga,1994-01-26,H,174.14,76.75,A+,gcz1994@example.edu,...,Tepalcatepec,Michoacan,19.192056,-102.845396,-0600,America/Mexico_City,Administrador de gobierno local,CAZG940126HMNAUO10,82 81 94 2519 6,aqua
6,Ing.,Amalia,Lara,Hernandes,1928-09-11,M,160.53,59.20,A+,amalia_lara@example.io,...,Chucarit,Sonora,27.030161,-109.583929,-0700,America/Hermosillo,Voluntario,LAHA280911MSRAEM58,64 44 28 3704 4,maroon
7,Dr.,Julian,Rodríquez,Zamudio,1991-02-25,H,169.29,65.83,A-,jrodriquez@example.com,...,Tamalín,Veracruz,21.338193,-97.812827,-0600,America/Mexico_City,Gerente del centro de conferencias,ROZJ910225HVZOAU44,76 89 91 8719 6,purple
8,None,Armando,Briones,Tirado,1993-09-08,H,183.33,66.21,O+,armandobrionestirado@example.edu,...,Jalpa de Méndez,Tabasco,18.176831,-93.062967,-0600,America/Mexico_City,Nutriologo de animales,BRTA930908HTCRIR95,74 24 93 4491 5,aqua
9,Lic.,Yolanda,Zayas,Reynoso,1939-03-24,M,174.41,75.12,O+,yzr39@example.gob,...,Colonia Irrigación,Sonora,NaN,NaN,None,None,Director de Estrategia,ZARY390324MSRAEO17,93 67 39 5246 6,gray


In [29]:
# mas info en https://randomuser.me/documentation
url_api = 'https://randomuser.me/api?format=csv'
params = {
	'results': 100,
	#'nat': 'es',
	#'password': '8-16,lower,upper,number,special',
	'exc': 'nat,picture,id,login,registered'
}
query = ''
for k, v in params.items():
	if isinstance(v, int): v = str(v)
	query += '&' + k + '=' + v
url = url_api + query
people = pd.read_csv(url)
people.sample(8)

,gender,name.title,name.first,name.last,location.street.number,location.street.name,location.city,location.state,location.country,location.postcode,location.coordinates.latitude,location.coordinates.longitude,location.timezone.offset,location.timezone.description,email,dob.date,dob.age,phone,cell
33,female,Miss,Shenna,Van Ravenhorst,2915,Halteriastraat,Scheulder,Drenthe,Netherlands,17500,-15.8411,165.5078,-5:00,"Eastern Time (US & Canada), Bogota, Lima",shenna.vanravenhorst@example.com,1982-01-15T14:04:18.916Z,40,(026)-467-5304,(525)-688-0957
78,male,Mr,Mason,White,150,Wiri Station Road,Palmerston North,Wellington,New Zealand,84303,-84.5117,-152.2561,+10:00,"Eastern Australia, Guam, Vladivostok",mason.white@example.com,1984-01-22T08:59:50.427Z,38,(278)-670-2846,(137)-909-5200
71,female,Mrs,Inger,Aam,4256,Ørakerfaret,Slemsrud,Description,Norway,6871,-78.4872,121.0481,+1:00,"Brussels, Copenhagen, Madrid, Paris",inger.aam@example.com,1969-10-19T08:58:50.214Z,53,67000514,96911015
45,male,Mr,علی,سالاری,6913,جمهوری اسلامی,نجف‌آباد,هرمزگان,Iran,78077,-75.0953,69.0367,-11:00,"Midway Island, Samoa",aaly.slry@example.com,1986-07-18T07:21:40.800Z,36,062-02282599,0928-131-1938
55,female,Ms,Nanna,Petersen,7113,Stengårdsvej,Randers Nv,Midtjylland,Denmark,57406,36.0369,36.7946,+8:00,"Beijing, Perth, Singapore, Hong Kong",nanna.petersen@example.com,1984-10-01T14:22:00.382Z,38,95755048,44487998
24,male,Mr,Felix,Anderson,6654,Lake of Bays Road,Enterprise,Québec,Canada,J0C 8E1,5.9558,-179.8199,+2:00,"Kaliningrad, South Africa",felix.anderson@example.com,1984-04-07T15:14:45.550Z,38,937-488-7448,190-416-3300
83,female,Mrs,Blanca,Martin,2146,Calle de Ángel García,Madrid,Asturias,Spain,53533,-55.8710,-140.4980,-6:00,"Central Time (US & Canada), Mexico City",blanca.martin@example.com,1991-06-30T15:53:12.237Z,31,982-352-967,678-111-568
76,male,Mr,Óscar,da Rosa,3272,Rua Santa Rita,Barretos,Goiás,Brazil,92501,75.1005,-130.2087,+5:30,"Bombay, Calcutta, Madras, New Delhi",oscar.darosa@example.com,1983-06-05T14:34:37.349Z,39,(44) 6825-3100,(90) 8831-5559
